# PyTorch Transfer Learning
<img src="resources/Cover Images/Transfer Learning with PyTorch Boosting Model Performance Cover Image.png" width=450 height=300 />

You can find the full written tutorial, with all explanations and code on my website: [link to PyTorch Transfer Learning tutorial](https://datagy.io/pytorch-transfer-learning/).
## Loading a Pre-Trained Model

In [1]:
# Importing Libraries
import torch
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder

In [2]:
# Setting up a Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# Load the pre-trained ResNet-50 model
weights=models.ResNet50_Weights.DEFAULT
resnet50 = models.resnet50(weights=weights).to(device)

## Modifying Our Final Layer

In [4]:
# Printing the ResNet50 Model
print(resnet50)

# Returns:
# ResNet(
# ...
#   (layer4): Sequential(
#     (0): Bottleneck(
#       (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
#       (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#     ...
#   (fc): Linear(in_features=2048, out_features=1000, bias=True)
# )

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
# Modify the final classification layer for our specific task
num_classes = 2
resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, num_classes)

In [6]:
# Freeze earlier layers
for param in resnet50.parameters():
    param.requires_grad = False

unfrozen_layers = ['layer4', 'fc', 'avgpool']  
for name, param in resnet50.named_parameters():
    if any(layer_name in name for layer_name in unfrozen_layers):
        param.requires_grad = True

## Preparing Our Data

In [7]:
# Define your desired transform for the training data
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No additional transform for the test data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
# Creating our Training and Testing Dataset
data_dir = 'data/transfer-learning-flowers/'
dataset = ImageFolder(root=data_dir)

# Split the dataset into training and testing sets
train_dataset, test_dataset = random_split(dataset, [0.7, 0.3])

# Apply the transforms to the respective datasets
train_dataset.dataset.transform = train_transform
test_dataset.dataset.transform = test_transform

In [9]:
# Defining Training and Testing DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

## Preparing the Model and Training

In [10]:
# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop (optional, but recommended)
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for val_inputs, val_labels in test_loader:
            val_outputs = resnet50(val_inputs)
            _, predicted = torch.max(val_outputs, 1)
            total_samples += val_labels.size(0)
            total_correct += (predicted == val_labels).sum().item()

        accuracy = total_correct / total_samples
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy:.4f}')


Epoch [1/5], Validation Accuracy: 0.9519
Epoch [2/5], Validation Accuracy: 0.9852
Epoch [3/5], Validation Accuracy: 0.9926
Epoch [4/5], Validation Accuracy: 0.9852
Epoch [5/5], Validation Accuracy: 0.9889
